In [ ]:
#liberary
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
import time
import datetime
import numpy as np
import ipywidgets as widgets
import glob
import os

In [ ]:
tracker_path='D:\\06.FullBeacon\\PO tracker'
file_list=glob.glob(tracker_path+'/*.xlsm')
PO_track_table=pd.read_excel(file_list[0],sheet_name='Report',skiprows=[0,2],header=[0],usecols='A,D,G,H,I,J,K,L,AO,AR,BC,BD,BE,BF,BG,BH')
PO_track_table=PO_track_table.rename(columns = {'FB360 Commissioned':'Plan End Date','FB360 Commissioned.1':'Actual End Date'})

In [ ]:
Beacon_tracker=PO_track_table[(PO_track_table['Site Activity']=='Full Beacon') | (PO_track_table['Site Activity']=='2Step') | (PO_track_table['Site Activity']=='2Step Cold Build')]
Beacon_tracker

In [ ]:
#Site type
def Classify(x):
    if '2Step'not in x:
        step_name=x+'2Step'
        if step_name in np.array(Beacon_tracker['DU ID']):
            return 'Full'
        else:
            return 'Beacon only'
    elif '2Step' in x:
        beacon_name=x.replace("2Step", "")
        if beacon_name in np.array(Beacon_tracker['DU ID']):
            return 'Attached'
        else:
            return '2Step only'
Beacon_tracker['Classification']=[*map(Classify ,Beacon_tracker['DU ID'])]

In [ ]:
#Check site plan-current setting-2 months
cancel_point= pd.Timestamp.today() + pd.Timedelta(2, unit='M')
def cancel_type(x):
    if pd.isnull(x):
        return 'de-instructed'
    elif x > cancel_point:
        return 'Plan date postponed'
    else:
        return 'keep-going'
Beacon_tracker['Cancel_type']=[*map(cancel_type, Beacon_tracker['Plan End Date'])]

In [ ]:
Beacon_tracker.groupby('Classification').size()

In [ ]:
#Check PO is exist
Beacon_tracker['Active BTS I&C PO No.']=Beacon_tracker['Active BTS I&C PO No.'].apply(lambda x: np.nan if 'Cancel' in str(x) else x)
Beacon_tracker['Passive Partner Build PO']=Beacon_tracker['Passive Partner Build PO'].apply(lambda x: np.nan if 'Cancel' in str(x) else x)
Beacon_tracker['PO_exist']=(pd.notnull(Beacon_tracker['Active BTS I&C PO No.']) | pd.notnull(Beacon_tracker['Passive Partner Build PO'])) 
Beacon_tracker.groupby(['Cancel_type','Classification','PO_exist']).size()

In [ ]:
#Mark same site in Mysite column
def mysite(x):
    if '2Step'not in x:
        step_name=x+'2Step'
        if step_name in np.array(Beacon_tracker['DU ID']):
            return step_name
        else:
            return np.nan
    if '2Step' in x:
        beacon_name=x.replace("2Step", "")
        if beacon_name in np.array(Beacon_tracker['DU ID']):
            return beacon_name
        else:
            return np.nan
Beacon_tracker['Mysite']=Beacon_tracker['DU ID'].apply(mysite)

In [ ]:
#Mark same site status in Mysite column
Partner=Beacon_tracker.copy()
Partner.set_index('DU ID',inplace=True)
Beacon_tracker['Mysite']=Beacon_tracker['Mysite'].map(Partner['Cancel_type'])
Beacon_tracker

In [ ]:
#Find Site needs to be cancelled, 1. not going 2. have PO 3.related site is not going ethier 
logic1=(Beacon_tracker['Cancel_type']!='keep-going')
logic2=Beacon_tracker['PO_exist']
logic3=(Beacon_tracker['Mysite']!='keep-going')
Cancel_table=Beacon_tracker[logic1&logic2&logic3]
Cancel_table=Cancel_table.reset_index(drop=True)
Cancel_table

In [ ]:
#Final check, Attached should be same as Full
Cancel_table.groupby('Classification').size()

In [ ]:
#Remove attached site(PO is not on attached site)
logic4=(Cancel_table['Classification']!='Attached')
Cacel_list=Cancel_table[logic4].reset_index(drop=True)


In [ ]:
Cacel_list

In [ ]:
writer = pd.ExcelWriter(r'D:\\06.FullBeacon\\Cancel_table2.xlsx')
Cancel_table.to_excel(writer,'Sheet1')
writer.save()